In [ ]:
#potentially alternative to uploading
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/dlam_project
!pwd

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
#!ls -lha kaggle.json
#!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
#!cp kaggle.json ~/.kaggle/ # move the key to the folder
#!pwd # checking the present working directory
#!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!pip install -q kaggle

In [ ]:
#%cd /content
#!kaggle datasets download arevel/chess-games
#!unzip -qq /content/chess-games.zip
#%cd /content

In [ ]:
#file_path = '/content/chess_games.csv'#
file_path = '/content/drive/My Drive/dlam_project/chess_games.csv'
#file_path = 'C:/Users/Nils/Downloads/archive/chess_games.csv'

In [1]:
!pip install chess

In [2]:
#IMPORTS
import numpy as np
import pandas as pd
import chess
import gc
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from neural_networks import CopiedChessNetwork
from datasets import Copied_Dataset


In [3]:
# SPECIFY DEFAULT SETTINGS
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_tensor_type(torch.DoubleTensor)

In [4]:
chess_data_raw = pd.read_csv(file_path, usecols=["AN","WhiteElo"])
chess_data= chess_data_raw[chess_data_raw['WhiteElo'] > 2000]
del chess_data_raw
gc.collect()
chess_data = chess_data[['AN']]
chess_data = chess_data[~chess_data['AN'].str.contains('{')]
chess_data = chess_data[chess_data['AN'].str.len() > 20]
print(chess_data.shape)


NameError: name 'file_path' is not defined

In [34]:
#board = chess.Board()
#dic = board.piece_map()
#for pos, piece in dic.items():
#    print(pos, piece.symbol())

#Gets the symbol P, N, B, R, Q or K for white pieces or the lower-case variants for the black pieces.

#board layout in storage
# 0 r n b q k b n r
# 1 p p p p p p p p
# 2 . . . . . . . .
# 3 . . . . . . . .
# 4 . . . . . . . .
# 5 . . . . . . . .
# 6 P P P P P P P P
# 7 R N B Q K B N R

# board layout in chess
# 8 r n b q k b n r
# 7 p p p p p p p p
# 6 . . . . . . . .
# 5 . . . . . . . .
# 4 . . . . . . . .
# 3 . . . . . . . .
# 2 P P P P P P P P
# 1 R N B Q K B N R
#   a b c d e f g h

# pos in piece_map is letter_tonum[letter] + 8 * number

In [ ]:
#data_train = Copied_Dataset(chess_data.iloc[:N_train], N_train)
#data_loader_train = DataLoader(data_train, batch_size = 32, shuffle = False)
#
#data_test = Copied_Dataset(chess_data.iloc[N_train: N_train + N_test], N_test)
#data_loader_test = DataLoader(data_test, batch_size = 32, shuffle = False)
N_train = 2000
N_test = 500
N_epochs = 5

data_train = Copied_Dataset(chess_data, N_train)
data_loader_train = DataLoader(data_train, batch_size = 64, shuffle = True)

data_test = Copied_Dataset(chess_data, N_test)
data_loader_test = DataLoader(data_test, batch_size = 64, shuffle = True)

# Training and Eval Loop

In [26]:
lr = 3e-4
hidden_size = 128
hidden_layers = 4
model = CopiedChessNetwork(hidden_layers=hidden_layers, hidden_size=hidden_size)
metric_from = nn.CrossEntropyLoss()
metric_to = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)


In [ ]:
model.eval()
for data, target in data_loader_test:
    losses = []
    with torch.no_grad():
        output = model(data)
        loss_from = metric_from(output[:, 0, :], target[:, 0, :])
        loss_to = metric_to(output[:, 1, :], target[:, 1, :])
        loss = loss_from + loss_to
        losses.append(loss)
print(np.mean(losses))


for i in range(N_epochs):
  model.train()

  for data, target in data_loader_train:
      model.zero_grad()
      pred = model(data)
      output = model(data)
      loss_from = metric_from(output[:, 0, :], target[:, 0, :])
      loss_to = metric_to(output[:, 1, :], target[:, 1, :])
      loss = loss_from + loss_to
      loss.backward()
      optimizer.step()

  model.eval()


  for data, target in data_loader_test:
      losses = []
      with torch.no_grad():
          output = model(data)
          loss_from = metric_from(output[:, 0, :], target[:, 0, :])
          loss_to = metric_to(output[:, 1, :], target[:, 1, :])
          loss = loss_from + loss_to
          losses.append(loss)

  print(np.mean(losses))

0.5181306407220865
0.18460245873644307
0.2285778128570642
0.1166661434569104
0.11449397977073866
0.1421459252988675
0.2263416419319615
0.15684008007116937
0.17551830675001562
0.15595597739577566
0.10708319070045116


In [27]:
torch.save(model.state_dict(), 'model_weights.pth')
#to load the weights again use:
#model = CopiedChessNetwork(hidden_layers=hidden_layers, hidden_size=hidden_size)
#model.load_state_dict(torch.load('model_weights_0dot15_loss.pth'))


<All keys matched successfully>